In [ ]:
from dotenv import load_dotenv  
load_dotenv(override=True)
import os, datetime as dt, json
import numpy as np
import pandas as pd
import yfinance as yf
from openai import OpenAI


# ----------------- Config -----------------
TICKER = os.environ.get("TICKER", "ETH-USD") # looks for an enviroment variable or defaults to "ETH-USD"
LOOKBACK_DAYS = int(os.environ.get("LOOKBACK_DAYS", "300"))
MODEL_ANALYST = os.environ.get("MODEL_ANALYST", "gpt-4o-mini")  # LLM-1
MODEL_AUDITOR = os.environ.get("MODEL_AUDITOR", "gpt-4.1-mini") # LLM-2
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

def ema(s, n): return s.ewm(span=n, adjust=False).mean()

def rsi(close, n=14):
    d = close.diff()
    up = d.clip(lower=0).rolling(n).mean()
    dn = (-d.clip(upper=0)).rolling(n).mean()
    rs = up / dn.replace(0, np.nan)
    return 100 - 100 / (1 + rs)

def atr(df, n=14):
    h, l, c = df["High"], df["Low"], df["Close"]
    pc = c.shift(1)
    tr = pd.concat([(h-l), (h-pc).abs(), (l-pc).abs()], axis=1).max(axis=1)
    return tr.rolling(n).mean()

def fetch_daily_eth():
    df = yf.download(TICKER, period="1y", interval="1d", auto_adjust=True, progress=False)

    if df.empty:
        raise SystemExit("No data fetched.")

    # If yfinance returns MultiIndex columns (e.g. ('Close','ETH-USD')), select this ticker
    if isinstance(df.columns, pd.MultiIndex):
        df = df.xs(TICKER, axis=1, level=1)

    # Now these are Series, not DataFrames
    df["ema20"] = ema(df["Close"], 20)
    df["ema50"] = ema(df["Close"], 50)
    df["rsi14"] = rsi(df["Close"], 14)
    df["atr14"] = atr(df, 14)
    df["atr_pct"] = (df["atr14"] / df["Close"]) * 100
    df["hh20"] = df["High"].rolling(20).max()
    df["ll20"] = df["Low"].rolling(20).min()
    df.dropna(inplace=True)
    return df

# ----------------- LLM helpers -----------------
def chat(model, system, user, temperature=0.1):
    resp = client.chat.completions.create(
        model=model, temperature=temperature,
        messages=[{"role":"system","content":system},
                  {"role":"user","content":user}]
    )
    return resp.choices[0].message.content.strip()


df = fetch_daily_eth()




user_analyst = (
        "Daily ETH metrics (use only these):\n"
        + json.dumps(metrics, indent=2)
        + "\nRespond with STRICT JSON only."
    )


user_analyst


IndentationError: unexpected indent (331876338.py, line 73)

In [25]:
# ----------------- Main -----------------
def main():
    df = fetch_daily_eth()
    latest = df.iloc[-1]
    prev = df.iloc[-2]

    date_utc = datetime.now(UTC).isoformat(timespec="seconds").replace("+00:00", "Z")

    metrics = {
        "ticker": TICKER,
        "date_utc": date_utc,
        "close": round(float(latest["Close"]), 2),
        "ema20": round(float(latest["ema20"]), 2),
        "ema50": round(float(latest["ema50"]), 2),
        "rsi14": round(float(latest["rsi14"]), 1),
        "atr_pct": round(float(latest["atr_pct"]), 2),
        "atr_abs": round(float(latest["atr14"]), 2),
        "hh20": round(float(latest["hh20"]), 2),
        "ll20": round(float(latest["ll20"]), 2),
        "prev_close": round(float(prev["Close"]), 2),
    }

    print("=== METRICS ===")
    print(json.dumps(metrics, indent=2))

    analyst, auditor, final = call_dual_llm(metrics)

    print("\n=== LLM-1 (Analyst) ===")
    print(json.dumps(analyst, indent=2))

    print("\n=== LLM-2 (Auditor) ===")
    print(json.dumps(auditor, indent=2))

    print("\n=== FINAL SIGNAL ===")
    print(json.dumps(final, indent=2))

    print("\nNote: Educational output only. Not financial advice.")


if __name__ == "__main__":
    main()


=== METRICS ===
{
  "ticker": "ETH-USD",
  "date_utc": "2025-08-29T16:57:20Z",
  "close": 4334.65,
  "ema20": 4377.2,
  "ema50": 3952.01,
  "rsi14": 47.8,
  "atr_pct": 6.12,
  "atr_abs": 265.25,
  "hh20": 4953.73,
  "ll20": 4064.97,
  "prev_close": 4507.18
}

=== LLM-1 (Analyst) ===
{
  "decision": "HOLD",
  "confidence": 0.5,
  "reasons": [
    "Close is below EMA20, indicating a potential downtrend.",
    "RSI14 is neutral, suggesting no strong momentum.",
    "ATR% indicates moderate volatility."
  ],
  "levels": {
    "entry": null,
    "stop": null,
    "target": null
  }
}

=== LLM-2 (Auditor) ===
{
  "verdict": "APPROVE",
  "correctedDecision": "HOLD",
  "notes": [
    "The close price is below the EMA20, suggesting a potential downtrend.",
    "RSI14 is neutral, indicating no strong momentum in either direction.",
    "ATR% shows moderate volatility, supporting a HOLD decision due to lack of clear trend."
  ],
  "finalLevels": {
    "entry": null,
    "stop": null,
    "target"